# Build tech tree
Grace Deng, August 2020 <br />
Updated by Natalia Velez, April 2021

In [10]:
import os, re, glob, json
from os.path import join as opj
import numpy as np
import pandas as pd
import json
import pymongo
from tqdm import notebook

# Embed html-formatted text
# (Used to do QA on transitions)
from IPython.core.display import display, HTML
def embed(s): return display(HTML(s))

## Load game data from database

Connect:

In [2]:
keyfile = '../6_database/credentials.key'

#Connection string
creds = open(keyfile, "r").read().splitlines()
myclient = pymongo.MongoClient('134.76.24.75', username=creds[0], password=creds[1], authSource='ohol') 
print(myclient)

ohol = myclient.ohol
print(ohol)

MongoClient(host=['134.76.24.75:27017'], document_class=dict, tz_aware=False, connect=True, authsource='ohol')
Database(MongoClient(host=['134.76.24.75:27017'], document_class=dict, tz_aware=False, connect=True, authsource='ohol'), 'ohol')


Objects:

In [3]:
objects = list(ohol.objects.find())
objects = pd.DataFrame(objects)

print('Loading %i objects' % objects.shape[0])
objects.head()

Loading 4161 objects


,_id,id,name,containable,containSize,vertSlotRot,permanent,minPickupAge,heldInHand,blocksWalking,...,floorHugging,slotsLocked,noFlip,sideAccess,creationSoundForce,invisCont,slotPos,vert,spritesDrawnBehind,spritesAdditiveBlend
0,60750c9e68dcd62afd70767e,11,Skin Tone A &B &C &D &E &F,0,1.0,0.0,0,3,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60750c9e68dcd62afd70767f,19,Female001 D,0,1.0,0.0,0,3,0,0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60750c9e68dcd62afd707680,30,Wild Gooseberry Bush,0,1.0,0.0,1,3,0,0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,60750c9e68dcd62afd707681,31,Gooseberry,1,1.0,0.0,0,3,1,0,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,60750c9e68dcd62afd707682,32,Big Hard Rock,0,1.0,0.0,1,3,0,0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Transitions:

In [4]:
transitions = list(ohol.expanded_transitions.find())
transitions = pd.DataFrame(transitions)

print('Loading %i transitions' % transitions.shape[0])
transitions.head()

Loading 19445 transitions


,_id,origActor,origTarget,newActor,newTarget,lastUseActor,lastUseTarget,prob
0,607aef65cffb881ebaafdd44,-1,1968,0,1969,False,False,True
1,607aef65cffb881ebaafdd45,-1,1968,0,1972,False,False,True
2,607aef65cffb881ebaafdd46,-1,1968,0,1970,False,False,True
3,607aef65cffb881ebaafdd47,-1,1968,0,1971,False,False,True
4,607aef65cffb881ebaafdd48,-1,1968,0,1973,False,False,True


Helper functions: Querying object, transition data

In [35]:
### Read from object data
def obj_name(i):
    if i in [0, -1]:
        return 'empty'
    else:
        return objects[objects.id == i].name.values[0]

### Read from transition data
def paths_from(i):
    '''
    Get all uses for object i
    '''
    return (transitions['origActor'] == i) | (transitions['origActor'] == i)

def paths_to(i):
    '''
    Get all paths that produce object i
    (When we build the tech tree, we're going to make the shortest of these paths
    the official 'recipe' for i and use it to calculate the depth of i) 
    '''
    is_product = (transitions['newActor'] == i) | (transitions['newTarget'] == i)
    is_ingredient = paths_from(i)    
    
    return is_product & (~is_ingredient)

def combinations(obj1, obj2):
    '''
    Gets all transitions involving obj1 and obj2, regardless of their relative positions
    '''
    obj1_actor = (transitions['origActor'] == obj1) & (transitions['origTarget'] == obj2)
    obj2_actor = (transitions['origActor'] == obj2) & (transitions['origTarget'] == obj1)   
    
    return obj1_actor | obj2_actor

### Display outputs nicely
trans_cols = ['origActor', 'origTarget', 'newActor', 'newTarget']
def record_transition(row):
    '''
    Converts row in transition df into a mongo-friendly dictionary
    '''
    return row[trans_cols].to_dict()
    

def display_transition(row):
    '''
    Displays transitions in pretty, HTML-formatted text
    '''
    # Get item names
    names = row[trans_cols].apply(obj_name)
    
    # Generate HTML tags
    tag = '<mark style="background-color: %s;">%s</mark>'
    actor_color = '#91d8f2'
    target_color = '#ffbf6b'
    out = '(%s,%s) &#x2192; (%s,%s)' % (tag % (actor_color, names['origActor']),
                                        tag % (target_color, names['origTarget']),
                                        tag % (actor_color, names['newActor']),
                                        tag % (target_color, names['newTarget']))
    
    embed(out)

Helper functions in action:

In [32]:
print(obj_name(72))

print('=== USES ===')
for _, row in transitions[paths_from(72)].iterrows():
    display_transition(row)

print('=== HOW TO GET ===')
for _, row in transitions[paths_to(72)].iterrows():
    display_transition(row)

Kindling
=== USES ===


=== HOW TO GET ===


## Search through Tech tree

Initialize Tech tree dictionary:

In [29]:
tech_dict = {o:None for o in objects.id.values}

Start with natural objects:

In [181]:
nat_objs = objects[objects.mapChance > 0].id.values.tolist()
# nat_objs = [32, 33, 63, 153, 50] # DEBUG: Constrained set of natural objects
print('== Starting from %i natural items ==' % len(nat_objs))
print(*['%s (%i)' %(obj_name(o), o) for o in nat_objs], sep='\n')

== Starting from 80 natural items ==
Wild Gooseberry Bush (30)
Big Hard Rock (32)
Stone (33)
Seeding Wild Carrot (36)
Juniper Tree (49)
Milkweed (50)
Maple Tree -Branch (63)
Lombardy Poplar Tree -Branch (65)
White Pine Tree (99)
White Pine Tree with Needles (100)
Tule Reeds (121)
Clay Deposit (125)
Flint (133)
Sapling (136)
Yew Tree -Branch (153)
Rabbit Hole -hiding &single (161)
Fertile Soil Deposit (211)
Ripe Wheat (242)
Flat Rock - empty (291)
Wolf (418)
Willow Tree (527)
Bald Cypress Tree (530)
Mouflon (531)
Bear Cave (630)
Limestone (674)
Gold Vein (680)
Penguin (703)
Ice Hole (706)
Antarctic Fur Seal (707)
Indigo (713)
Rose Madder (714)
Alum (729)
Dead Tree (760)
Barrel Cactus (761)
Rattle Snake (764)
Wild Horse (769)
Monolith (791)
Burdock (804)
Wild Onion (805)
Muddy Iron Vein - gridPlacement40 &40 (942)
Wild Rose with Fruit (1013)
Snow Bank (1020)
Teosinte (1107)
Wild Potato (1140)
Wild Bean Plant (1157)
Wild Squash Plant (1184)
Wild Cabbage (1203)
Canada Goose Pond with Egg (

Search iteratively through Tech tree

(Note: This is pretty inefficient, as we repeatedly test the same combinations - fix before scaling up?)

In [178]:
# Initialize values
known_objs = [0, -1] + nat_objs
tech_tree = {o: {'depth': 0, 'recipe': None} for o in known_objs}
counter = 1
n_new = len(known_objs)
combos = [(o_i, o_j) 
          for i,o_i in enumerate(known_objs)
          for j,o_j in enumerate(known_objs[i:])]

while counter < 4:
    print('=== ITERATION %i ===' % counter)
    
    # Search for viable combinations
    product_list = [transitions[combinations(o_i, o_j)] for o_i, o_j in combos]
    product_df = pd.concat(product_list)
    
    # Which combinations yield new products?
    new_objs = []
    for _, row in product_df.iterrows():
        for e in ['newActor', 'newTarget']:
            product = row[e]
            
            # New products are added to the tech tree
            if product not in tech_tree.keys():
                print('%s (%i)' % (obj_name(product), product))
                new_objs.append(product)
                tech_tree[product] = {}
                tech_tree[product]['depth'] = counter
                tech_tree[product]['recipe'] = record_transition(row)
                
    # Prepare for next iteration
    counter += 1
    n_new = len(new_objs)
    
    # Update combinations in search space    
    if n_new > 0:
        combo_old = [(o_i, o_j) for o_i in known_objs for o_j in new_objs]
        combo_new = [(o_i, o_j) for i,o_i in enumerate(new_objs) for j,o_j in enumerate(new_objs[i:])]
        combos = combo_old + combo_new
        known_objs += new_objs
    else:
        print('(None found)')    

=== ITERATION 1 ===
Gooseberry (31)
Empty Wild Gooseberry Bush (279)
Carrot Seed Head (395)
Wild Carrot (404)
Juniper Tinder (61)
Milkweed Stalk (57)
Milkweed Debris (53)
Straight Branch (64)
Maple Tree (48)
Small Curved Branch (66)
Lombardy Poplar Tree (45)
Pine Needles (96)
Clay (126)
Clay Pit -partial (409)
Yew Branch (132)
Yew Tree (406)
Wheat Seed Head (223)
Attacking Wolf (427)
Bite Wound (1363)
Attacking Rattle Snake (1385)
Snake Bite (1377)
Wild Onion (808)
Wild Rose Hip (1015)
Wild Rose - picked (1014)
Snowball - +noWait (2407)
Teosinte Seed Head (1108)
Bare Teosinte (1123)
Dry Bean Pod (1160)
Empty Wild Bean Plant (1158)
Wild Squash (1183)
Empty Wild Squash Plant (1185)
Cold Goose Egg (1262)
Canada Goose Pond (141)
Attacking Wild Boar (1333)
Hog Cut (1364)
Mango Leaf (1878)
Banana (2143)
Empty Banana Plant (2145)
Mosquito Swarm -just bit (2157)
Yellow Fever - sick emot_7_35  food_0.3 fever_15 (2155)
Wild Tomato Cluster (2801)
Empty Wild Tomato Plant (2803)
Wild Pepper (2807)


In [180]:
tech_tree[2143]

{'depth': 1,
 'recipe': {'origActor': 0,
  'origTarget': 2142,
  'newActor': 2143,
  'newTarget': 2142}}

In [177]:
tech_tree[72]

{'depth': 6,
 'recipe': {'origActor': 71,
  'origTarget': 64,
  'newActor': 71,
  'newTarget': 72}}